# はじめに

本ノートブックでは、[Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic) を題材として、データサイエンスの基礎を学ぶ。
基本的な流れは、[Spaceship Titanic: A complete guide](https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide) を参考させてもらっているが、（データサイエンスを学ぶことも1つの目的であるため）細かい記載の修正や解説などが追記されていることがある。

In this notebook, we will learn the basics of data science in Japanese using the [Spaceship Titanic](https://www.kaggle.com/competitions/spaceship-titanic) as a subject. The basic flow is referenced from [Spaceship Titanic: A complete guide](https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide), but there may be minor revisions and additional explanations to aid in learning data science.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ライブラリ

まず初めに、データサイエンスで用いる基本的なライブラリをimportする。

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style = 'darkgrid', font_scale = 1.4)

以下では、警告を無視する設定を追加する。例えば、下記コード内における `sns.histplot` では、PandasとSeabornのバージョンの互換性に関する警告（将来のバージョンで、非推奨となる機能が含まれている）が出るが、ここでは（本質的でないと判断し）抑制する。

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

何はともあれ、予測するためにはモデルを使用する必要がある。どのモデルを使うと良いか判断する必要があるため、いくつかここで準備し、後で、望ましいモデルを選択する。

追記：それぞれの簡単な解説

In [ ]:
# ロジスティック回帰モデル
from sklearn.linear_model import LogisticRegression

# k最近傍法
from sklearn.neighbors import KNeighborsClassifier

# サポートベクトルマシン
from sklearn.svm import SVC

# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier

# LigthGBM（勾配ブースティング）
from lightgbm import LGBMClassifier

# CatBoost
from catboost import CatBoostClassifier

# Gaussianナイーブベイズ
from sklearn.naive_bayes import GaussianNB

# データの基本的な部分を確認

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

# (行数, 列数)
print('train.shape:', train.shape)
print('test.shape:', test.shape)

In [ ]:
train.head()

# 欠損値の確認

なぜ、欠損値を確認するか？

In [ ]:
# trainの欠損値
train.isna().sum()

In [ ]:
# testの欠損値
test.isna().sum()

# 重複行の確認

なぜ、重複行を確認するか

In [ ]:
train.duplicated().sum()

In [ ]:
test.duplicated().sum()

# 特徴量について

特徴量は、以下の3つに分類できる：

- continuous
- categorical
- descriptive / qualitative

| 特徴量|aaa |
|---|--- |
| PassengerId | descriptive / qualitative |
| HomePlanet |categorical |
| CryoSleep |categorical |
| Cabin | descriptive / qualitative |
| Destination |categorical |
| Age |continuous |
| VIP | categorical |
| RoomService |continuous |
| FoodCourt |continuous |
| ShoppingMail |continuous |
| Spa | continuous|
| VRDeck |continuous |
| Name |descriptive / qualitative |

In [ ]:
# datatypeの確認
train.dtypes

# 探索的データ解析

EDA（Exploratory Data Analysis）

In [ ]:
# 図のサイズ
plt.figure(figsize=(6, 6))

train['Transported'].value_counts().plot.pie(
    autopct = '%1.1f%%'
).set_title("Target distribution")

上図から、半分半分ということがわかる

乗客の年齢分布について視覚化する

In [ ]:
plt.figure(figsize = (10, 4))

sns.histplot(data=train, x ='Age', hue='Transported', binwidth=1, kde = True)

plt.title('Age distribution')
plt.xlabel('Age (years)')

0-18, 18-25, 25以上で、傾向が読み取れる

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
→この辺は、金銭系なので、それで、まとめて傾向を見る
→傾向が似ていれば、1つの特徴量を作れる

...後で加筆するかも

Categorical featuresについて

- 傾向あり
- 特徴量から落とすこともできる

### Feature engineering

#### 年齢について

新しい特徴量である `Age_group` を定義する

In [ ]:
# train
# Age_groupを定義し、空文字で初期化する
# np.nanによる初期化だと、データタイプの不整合が起きる
train['Age_group'] = ''

train.loc[train['Age'] <= 12, 'Age_group'] = 'Age_0-12'
train.loc[(train['Age'] > 12) & (train['Age'] < 18), 'Age_group'] = 'Age_13-17'
train.loc[(train['Age'] >= 18 ) & (train['Age'] <= 25), 'Age_group'] = 'Age_18-25'
train.loc[(train['Age'] > 25 ) & (train['Age'] <= 30), 'Age_group'] = 'Age_26-30'
train.loc[(train['Age'] >= 30 ) & (train['Age'] <= 50), 'Age_group'] = 'Age_31-50'
train.loc[train['Age'] > 50, 'Age_group'] = 'Age_51+'

In [ ]:
# test
test['Age_group'] = ''

test.loc[test['Age'] <= 12, 'Age_group'] = 'Age_0-12'
test.loc[(test['Age'] > 12) & (test['Age'] < 18), 'Age_group'] = 'Age_13-17'
test.loc[(test['Age'] >= 18 ) & (test['Age'] <= 25), 'Age_group'] = 'Age_18-25'
test.loc[(test['Age'] > 25 ) & (test['Age'] <= 30), 'Age_group'] = 'Age_26-30'
test.loc[(test['Age'] >= 30 ) & (test['Age'] <= 50), 'Age_group'] = 'Age_31-50'
test.loc[test['Age'] > 50, 'Age_group'] = 'Age_51+'

In [ ]:
# 作成した特徴量に対して、グラフ化する
plt.figure(figsize=(10, 4))
g = sns.countplot(
    data = train,
    x = 'Age_group',
    hue = 'Transported',
    order = [
        'Age_0-12',
        'Age_13-17',
        'Age_18-25',
        'Age_26-30',
        'Age_31-50',
        'Age_51+'
    ]
)
plt.title('Age group distribution')

Age_0-12 については、transportedされている数が、そうでないのに比べ、多い

### どのモデルを使うか

ここでは複数の機械学習モデルを、（モデル名をkey、モデルのインスタンスをvalueとした）辞書形式で定義している。

In [ ]:
classifiers = {
    "LogisticRegression" : LogisticRegression(random_state=0),
    "KNN" : KNeighborsClassifier(),
    "SVC" : SVC(random_state=0, probability=True),
    "RandomForest" : RandomForestClassifier(random_state=0),
    "LGBM" : LGBMClassifier(random_state=0),
    "CatBoost" : CatBoostClassifier(random_state=0, verbose=False),
    "NaiveBayes" : GaussianNB()
}

ハイパーパラメータを設定する

grid searchについて追記

In [ ]:
LR_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.25, 0.5, 0.75, 1, 1.25, 1.5],
    'max_iter' : [50, 100, 150]
}

KNN_grid = {
    'n_neighbors' : [3, 5, 7, 9],
    'p' : [1, 2]
}

SVC_grid = {
    'C' : [0.25, 0.5, 0.75, 1, 1.25, 1.5],
    'kernel' : ['linear', 'rbf'],
    'gamma' : ['scale', 'auto']
}

RF_grid = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'max_depth' : [4, 6, 8, 10, 12]
}

boosted_grid = {
    'n_estimators' : '50, 100, 150, 200',
    'max_depth' : [4, 8, 12],
    'learning_rate' : [0.05, 0.1, 0.15]
}

NB_grid = {
    'var_sommthing' : [1e-10, 1e-9, 1e-8, 1e-7]
}

In [ ]:
grid = {
    "LogisticRegression" : LR_grid,
    "KNN" : KNN_grid,
    "SVC" : SVC_grid,
    "RandomForest" : RF_grid,
    "XGBoost" : boosted_grid,
    "LGBM" : boosted_grid,
    "CatBoost" : boosted_grid,
    "NaiveBayes" : NB_grid
}

In [ ]:
i = 0